In [1]:
import os
# 必须在 import transformers 或 pipeline 之前运行
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
import pandas as pd
data = pd.read_csv("ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [4]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [5]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]

    def __len__(self):
        return len(self.data)

In [6]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


In [7]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [8]:
for i in range(5):
    print(trainset[i])

('我住的是虹口商务酒店，就在虹口大酒点的对面，相比较起来外观实在落差很大，酒店大堂也小的可怜但是入住之后才发现不错。优点：价格适合年轻人或居家旅行，性价比比较高。卧房和卫生间干净整洁，地理位置紧邻滨海大道（烟台人的骄傲，很漂亮），交通便利，环境很好。缺点：房间隔音比较差，内部设备有些简单陈旧（电视），大堂很小，和对面的虹口大酒店对比落差很大，并且大堂内常有鸡出入，很是不爽。', 1)
('我是7月22日入住酒店的,酒店位于市中心,交通非常方便.酒店规模很大，较多会议接待及商务客人，是真正的商务型酒店；房间相当舒适、干净，而且让人感觉很安全；酒店软件很不错，服务员非常热情周到；自助早餐很丰富，除了可口的点心外还有一些当地特产的木耳和黑笋......酒店宽带及长途电话是免费的，对于我们这种商务客人来说非常方便，入住当天还有水果赠送，大都是当地产的，味道很鲜美。销售部的工作人员服务也非常好，我们是次日凌晨六点退的房，他们特意提前为我们打包好了早餐，我们真的挺感动的，总之，真的是物超所值，下次来还会选择这里！', 1)
('不错的酒店，服务很好，总台主管不仅帮我们做了免费升级，还亲自带我们上房间。房间很大，也很舒服，有很大的液晶显示屏，并且有免费饮料和小点心。刚进房间不久就有服务员泡来咖啡，尤其是离开的时候，服务员还帮我们拿了些饮料路上喝，并且还准备了两份报纸。总之来说，服务意识很强，让人感觉很贴心。很满意！唯一有点美中不足的是，有一间房电视信号不是很稳定，不是很清楚，希望能加以改善。下次还会选择入住这里。', 1)
('说是9层在装修！但是住5层还是吵得睡不着。怀疑6层也在装修，声音就在头顶。电钻6、7点就开始吵。头痛欲裂。地点很不好找。不临街。宾馆反馈2008年6月2日：尊敬的客人：您好！您在入住期间未能享受到安静的环境，我们深感不安，并向您表示深切的歉意。为了创造酒店优美环境，使广大宾客居住环境更加舒适，我店正在进行客房硬件的装修改造工作，施工时间为8：00－20：00，非常抱歉施工单位未在规定时间内施工，还打扰了您的休息，为此，我们已经与施工方再次接洽，要求他们遵守工作时间，杜绝此类事件的再次发生。酒店楼顶正装修并安装本市最具特色的标志建筑：“金葫芦”，相信您再次光临本店时会给您焕然一新的感觉，更会让您感受到我们更热诚的服务。再次谢谢您的忠肯评价，并真诚地向您表示

In [9]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [10]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [11]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101,  857, 6814,  ...,    0,    0,    0],
        [ 101,  817, 3419,  ...,    0,    0,    0],
        [ 101, 4692,  749,  ...,  671,  702,  102],
        ...,
        [ 101, 3302, 1218,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 1

In [12]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")
optimizer = Adam(model.parameters(), lr=2e-5)
if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

In [14]:
train()

ep: 0, global_step: 0, loss: 0.8491284847259521
ep: 0, global_step: 100, loss: 0.4284001290798187
ep: 0, global_step: 200, loss: 0.19214290380477905
ep: 0, acc: 0.8891752362251282
ep: 1, global_step: 300, loss: 0.1478845179080963
ep: 1, global_step: 400, loss: 0.21112528443336487
ep: 1, acc: 0.9020618200302124
ep: 2, global_step: 500, loss: 0.21461288630962372
ep: 2, global_step: 600, loss: 0.07243625819683075
ep: 2, acc: 0.9046391248703003


In [15]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [16]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [17]:
pipe(sen)

[{'label': '好评！', 'score': 0.991304874420166}]